# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
city_data_to_load = "city_data_SP.csv"
city_data = pd.read_csv(city_data_to_load)

city_data

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lagoa,39.05,-27.98,72.90,88,0,5.68,PT,1595905426
1,1,Punta Arenas,-53.15,-70.92,34.43,59,61,8.05,CL,1595905426
2,2,Butaritari,3.07,172.79,83.17,78,100,11.54,KI,1595905426
3,3,Sisimiut,66.94,-53.67,43.92,94,100,6.64,GL,1595905426
4,4,Los Llanos de Aridane,28.66,-17.92,71.94,73,90,17.22,ES,1595905426
...,...,...,...,...,...,...,...,...,...,...
530,548,Kinablangan,7.69,126.55,85.37,64,60,4.54,PH,1595905688
531,549,Sivas Province,39.79,37.00,59.00,58,0,2.24,TR,1595905688
532,550,Ahipara,-35.17,173.17,60.55,66,58,10.45,NZ,1595905689
533,551,Oudtshoorn,-33.59,22.20,48.20,76,0,3.36,ZA,1595905689


In [20]:
city_data.dtypes

Unnamed: 0       int64
City            object
Latitude       float64
Longitude      float64
Temperature    float64
Humidity         int64
Cloudiness       int64
Wind Speed     float64
Country         object
Date             int64
dtype: object

In [21]:
city_data.describe()

,Unnamed: 0,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
count,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,5.350000e+02
mean,276.760748,19.804056,17.424056,68.863589,71.011215,50.014953,7.856206,1.595906e+09
std,159.722755,33.024838,91.568152,13.291304,19.103990,39.784153,5.396880,1.420532e+02
min,0.000000,-54.800000,-179.170000,32.000000,3.000000,0.000000,0.380000,1.595905e+09
25%,138.500000,-7.425000,-62.720000,59.280000,59.500000,1.000000,3.840000,1.595905e+09
50%,277.000000,22.570000,20.870000,71.600000,76.000000,52.000000,6.460000,1.595906e+09
75%,415.500000,46.520000,101.290000,79.730000,85.000000,90.000000,11.350000,1.595906e+09
max,552.000000,78.220000,179.320000,98.020000,99.000000,100.000000,28.860000,1.595906e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Store latitude and longitude in locations
locations = city_data[["Latitude", "Longitude"]]
hum_data = city_data["Humidity"].astype(float)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum_data, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Ideal Conditions
#A max temperature lower than 73
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [25]:
city_data.drop(city_data[city_data["Temperature"] > 80].index, inplace = True)
city_data.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lagoa,39.05,-27.98,72.90,88,0,5.68,PT,1595905426
1,1,Punta Arenas,-53.15,-70.92,34.43,59,61,8.05,CL,1595905426
3,3,Sisimiut,66.94,-53.67,43.92,94,100,6.64,GL,1595905426
4,4,Los Llanos de Aridane,28.66,-17.92,71.94,73,90,17.22,ES,1595905426
5,5,Margate,51.38,1.39,60.66,72,32,1.99,GB,1595905382


In [26]:
city_data.drop(city_data[city_data["Wind Speed"] > 10].index, inplace = True)
city_data.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lagoa,39.05,-27.98,72.90,88,0,5.68,PT,1595905426
1,1,Punta Arenas,-53.15,-70.92,34.43,59,61,8.05,CL,1595905426
3,3,Sisimiut,66.94,-53.67,43.92,94,100,6.64,GL,1595905426
5,5,Margate,51.38,1.39,60.66,72,32,1.99,GB,1595905382
7,7,East London,-33.02,27.91,55.81,59,0,5.82,ZA,1595905203


In [32]:
city_data.drop(city_data[city_data["Cloudiness"] > 0].index, inplace = True)
city_data.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lagoa,39.05,-27.98,72.90,88,0,5.68,PT,1595905426
7,7,East London,-33.02,27.91,55.81,59,0,5.82,ZA,1595905203
11,11,Port Alfred,-33.59,26.89,55.42,62,0,7.23,ZA,1595905305
17,17,Bredasdorp,-34.53,20.04,39.20,93,0,1.12,ZA,1595905427
20,20,Qaanaaq,77.48,-69.36,40.17,70,0,0.94,GL,1595905427


In [36]:
city_data.count()

Unnamed: 0     65
City           65
Latitude       65
Longitude      65
Temperature    65
Humidity       65
Cloudiness     65
Wind Speed     65
Country        58
Date           65
dtype: int64

In [34]:
filtered_data = city_data[city_data['Country'].notna()]

In [35]:
filtered_data.count()

Unnamed: 0     58
City           58
Latitude       58
Longitude      58
Temperature    58
Humidity       58
Cloudiness     58
Wind Speed     58
Country        58
Date           58
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
hotel_df = pd.DataFrame(filtered_data, columns=["City", "Country", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Lagoa,PT,39.05,-27.98,
7,East London,ZA,-33.02,27.91,
11,Port Alfred,ZA,-33.59,26.89,
17,Bredasdorp,ZA,-34.53,20.04,
20,Qaanaaq,GL,77.48,-69.36,
33,Senanga,ZM,-16.12,23.27,
36,Sadova,RO,43.90,23.95,
53,Hermanus,ZA,-34.42,19.23,
78,Port Elizabeth,ZA,-33.92,25.57,
92,Oktyabr'skoye,RU,43.06,44.74,


In [66]:
target_coordinates = f"{hotel_df['Latitude'][0]},{hotel_df['Longitude'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [64]:
hotel_name = []


params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    
    name_address = name_address.json()
    
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [65]:
hotel_name

['Graciosa Resort - Biosphere Island Hotel', 'Kennaway Hotel', 'The Halyards Hotel', 'De Hoop Collection', 'Qaanaaq Hotel', 'Kashuwa Lodge', nan, 'The Marine Hermanus', 'Lemon Tree Lane Guest House', 'Aleksandrovskiy', 'Red Lion Inn & Suites Polson', 'Akhavan Hotel', 'Desert Gat', 'NH La Perdiz', 'Beringgorski', 'hôtel paradis de Tshikapa', 'Gostinitsa "Zapolyar\'ye"', 'Berlevåg camping & Appartement AS', 'Residencial Hambyll', 'Protea Hotel by Marriott Saldanha Bay', 'Hotel Mapuera', nan, 'Jalu Hotel', nan, 'Quality Suites Vila Velha', 'Hotel Darya', 'Niriides Villas Hotel', 'Free Zone Hotel Nouadhibou', nan, 'Апартаменты «Дом 213»', 'Hotel casa real', 'VicTori', 'Phofu Eco Safari Lodge', 'Pier Resort', nan, 'Sofía Soberana Hotel Boutique', 'Hotel Devon', 'Gostinitsa Siberia', 'Doosti Hotel', 'Hotel Restaurante Juanito', 'Gornitsa', 'Los Tajibos Hotel & Convention Center', 'Cabañas Los Alamos I', nan, 'Hotel Chik Chik Namibe', 'DoubleTree by Hilton Hotel Anshun', 'Casa Ládico Boutique

In [67]:
hotel_df['Hotel Name']= hotel_name
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Lagoa,PT,39.05,-27.98,Graciosa Resort - Biosphere Island Hotel
7,East London,ZA,-33.02,27.91,Kennaway Hotel
11,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
17,Bredasdorp,ZA,-34.53,20.04,De Hoop Collection
20,Qaanaaq,GL,77.48,-69.36,Qaanaaq Hotel
33,Senanga,ZM,-16.12,23.27,Kashuwa Lodge
36,Sadova,RO,43.90,23.95,NaN
53,Hermanus,ZA,-34.42,19.23,The Marine Hermanus
78,Port Elizabeth,ZA,-33.92,25.57,Lemon Tree Lane Guest House
92,Oktyabr'skoye,RU,43.06,44.74,Aleksandrovskiy


In [68]:
hotel_df[hotel_df['Hotel Name'].notna()]

,City,Country,Latitude,Longitude,Hotel Name
0,Lagoa,PT,39.05,-27.98,Graciosa Resort - Biosphere Island Hotel
7,East London,ZA,-33.02,27.91,Kennaway Hotel
11,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
17,Bredasdorp,ZA,-34.53,20.04,De Hoop Collection
20,Qaanaaq,GL,77.48,-69.36,Qaanaaq Hotel
33,Senanga,ZM,-16.12,23.27,Kashuwa Lodge
53,Hermanus,ZA,-34.42,19.23,The Marine Hermanus
78,Port Elizabeth,ZA,-33.92,25.57,Lemon Tree Lane Guest House
92,Oktyabr'skoye,RU,43.06,44.74,Aleksandrovskiy
102,Polson,US,47.69,-114.16,Red Lion Inn & Suites Polson


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [72]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))